# Алгоритм градиентного бустинга

--------
## Задача классификации

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [31]:
path = 'winequality-red.csv'
data_class = pd.read_csv(path)

print("Первые 5 строк датасета для классификации:")
print(data_class.head())
print(f"\nРазмер датасета: {data_class.shape}")

Первые 5 строк датасета для классификации:
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  

### Создаем целевую переменную для классификации

In [32]:
data_class['wine_quality'] = data_class['quality'].apply(lambda x: 'good' if x >= 6 else 'bad')
data_class = data_class.drop('quality', axis=1)

print("\nРаспределение качества вина:")
print(data_class['wine_quality'].value_counts())


Распределение качества вина:
wine_quality
good    855
bad     744
Name: count, dtype: int64


### Разделим датасет на признаки и целевую переменную и масштабирование

In [33]:
X = data_class.drop('wine_quality', axis=1)
y = data_class['wine_quality']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

print(f"Размер обучающей выборки: {X_train.shape}, Размер тестовой выборки: {X_test.shape}")

Размер обучающей выборки: (1279, 11), Размер тестовой выборки: (320, 11)


### Применим встроенный алгоритм для градиентного бустинга

In [34]:
gb_baseline = GradientBoostingClassifier(random_state=42)
gb_baseline.fit(X_train, y_train)

y_pred_baseline = gb_baseline.predict(X_test)

accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
f1_baseline = f1_score(y_test, y_pred_baseline, average='weighted')

print("Бейзлайн (Gradient Boosting):")
print(f"Accuracy: {accuracy_baseline:.4f}")
print(f"F1-Score: {f1_baseline:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_baseline))

Бейзлайн (Gradient Boosting):
Accuracy: 0.7906
F1-Score: 0.7909

Classification Report:
              precision    recall  f1-score   support

         bad       0.75      0.82      0.78       149
        good       0.83      0.77      0.80       171

    accuracy                           0.79       320
   macro avg       0.79      0.79      0.79       320
weighted avg       0.79      0.79      0.79       320



### Подберем гиперпараметры для улучшения бейзлайна

##### `n_estimators: [50, 100, 200]`
- **Количество деревьев в ансамбле** - баланс между точностью и временем обучения
- **50**: быстрая тренировка, подходит для начальной оценки
- **100**: стандартный выбор, хороший баланс производительности
- **200**: больше деревьев для потенциально лучшей точности (риск переобучения)

##### `learning_rate: [0.01, 0.1, 0.2]`
- **Темп обучения** - определяет вклад каждого дерева
- **0.01**: медленное обучение, требует больше деревьев, но более точное
- **0.1**: стандартное значение, хороший баланс скорости и точности
- **0.2**: более агрессивное обучение, быстрее сходится

##### `max_depth: [3, 5, 7]`
- **Глубина деревьев** - контроль сложности weak learners
- **3**: очень простые деревья (пни), высокая bias
- **5**: умеренная сложность, стандартный выбор
- **7**: более сложные деревья, лучше захватывают паттерны (риск переобучения)

##### `subsample: [0.8, 1.0]`
- **Доля samples для каждого дерева** - Стохастический градиентный бустинг
- **0.8**: 80% данных для каждого дерева, увеличивает разнообразие
- **1.0**: все данные (стандартный бустинг)

##### `min_samples_split: [2, 5, 10]`
- **Минимальные samples для разделения узлов**
- **2**: максимальная детализация
- **5**: умеренное ограничение
- **10**: сильное ограничение, предотвращает переобучение

##### Ключевые взаимодействия параметров:
- **n_estimators × learning_rate**: компромисс - меньший learning_rate требует больше деревьев
- **max_depth × learning_rate**: более глубокие деревья с малым learning_rate
- **subsample**: добавляет случайность, как в Random Forest

##### Общая стратегия:
Поиск оптимального баланса между силой отдельных деревьев (max_depth) и общим ансамблем (n_estimators, learning_rate) с регуляризацией через subsampling и ограничения разделения.

In [35]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(
    GradientBoostingClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Лучшие параметры: {best_params}")

Fitting 5 folds for each of 162 candidates, totalling 810 fits
Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 0.8}


### Проведем обучение улучшенной модели и посчитаем метрики

In [36]:
gb_best = grid_search.best_estimator_

y_pred_best = gb_best.predict(X_test)

accuracy_best = accuracy_score(y_test, y_pred_best)
f1_best = f1_score(y_test, y_pred_best, average='weighted')

print("Улучшенный бейзлайн (Gradient Boosting):")
print(f"Accuracy: {accuracy_best:.4f}")
print(f"F1-Score: {f1_best:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best))

Улучшенный бейзлайн (Gradient Boosting):
Accuracy: 0.8281
F1-Score: 0.8283

Classification Report:
              precision    recall  f1-score   support

         bad       0.79      0.86      0.82       149
        good       0.87      0.80      0.83       171

    accuracy                           0.83       320
   macro avg       0.83      0.83      0.83       320
weighted avg       0.83      0.83      0.83       320



### Реализуем свой алгоритм

In [37]:
class CustomGradientBoosting:
    def __init__(self, n_estimators=10, learning_rate=0.1, max_depth=3, min_samples_split=2, min_samples_leaf=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.models = []
        self.learning_rate = learning_rate

    def fit(self, X, y):
        self.classes = np.unique(y)
        y_encoded = np.array([self.classes.tolist().index(label) for label in y])
        n_classes = len(self.classes)
        self.models = []
        F_m = np.zeros((X.shape[0], n_classes))
        
        for i in range(self.n_estimators):
            gradient = np.zeros((X.shape[0], n_classes))
            for k in range(n_classes):
                indicator = (y_encoded == k).astype(int)
                probability = self._softmax(F_m[:, k])
                gradient[:, k] = indicator - probability

            tree = DecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                random_state=np.random.randint(0, 10000)
            )
            tree.fit(X, gradient.argmax(axis=1))
            self.models.append(tree)

            for k in range(n_classes):
                F_m[:, k] += self.learning_rate * tree.predict_proba(X)[:, k]

    def _softmax(self, x):
        exp_x = np.exp(x - np.max(x))
        return exp_x / exp_x.sum(axis=0)

    def predict(self, X):
        F_m = np.zeros((X.shape[0], len(self.classes)))

        for tree in self.models:
            tree_probs = tree.predict_proba(X)
            for k in range(len(self.classes)):
                F_m[:, k] += self.learning_rate * tree_probs[:, k]

        y_pred_indices = np.argmax(F_m, axis=1)
        return np.array([self.classes[i] for i in y_pred_indices])

### Посмотрим на результат

In [38]:
custom_gb = CustomGradientBoosting(
    n_estimators=50,
    learning_rate=0.1,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1
)
custom_gb.fit(X_train, y_train)

y_pred_custom_gb = custom_gb.predict(X_test)

accuracy_custom_gb = accuracy_score(y_test, y_pred_custom_gb)
f1_custom_gb = f1_score(y_test, y_pred_custom_gb, average='weighted')

print("Custom Gradient Boosting (базовый):")
print(f"Accuracy: {accuracy_custom_gb:.4f}")
print(f"F1-Score: {f1_custom_gb:.4f}")

custom_gb_improved = CustomGradientBoosting(
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=1
)
custom_gb_improved.fit(X_train, y_train)

y_pred_custom_gb_improved = custom_gb_improved.predict(X_test)

accuracy_custom_gb_improved = accuracy_score(y_test, y_pred_custom_gb_improved)
f1_custom_gb_improved = f1_score(y_test, y_pred_custom_gb_improved, average='weighted')

print("Custom Gradient Boosting (улучшенный):")
print(f"Accuracy: {accuracy_custom_gb_improved:.4f}")
print(f"F1-Score: {f1_custom_gb_improved:.4f}")


Custom Gradient Boosting (базовый):
Accuracy: 0.7063
F1-Score: 0.7006
Custom Gradient Boosting (улучшенный):
Accuracy: 0.7250
F1-Score: 0.7253


### Сравним результат

In [39]:
print("\nСравнение результатов для задачи классификации:")
print(f"Бейзлайн Accuracy: {accuracy_baseline:.4f}, Улучшенный Accuracy: {accuracy_best:.4f}, Custom GB Accuracy: {accuracy_custom_gb:.4f}, Custom GB улучшенный Accuracy: {accuracy_custom_gb_improved:.4f}")
print(f"Бейзлайн F1-Score: {f1_baseline:.4f}, Улучшенный F1-Score: {f1_best:.4f}, Custom GB F1-Score: {f1_custom_gb:.4f}, Custom GB улучшенный F1-Score: {f1_custom_gb_improved:.4f}")


Сравнение результатов для задачи классификации:
Бейзлайн Accuracy: 0.7906, Улучшенный Accuracy: 0.8281, Custom GB Accuracy: 0.7063, Custom GB улучшенный Accuracy: 0.7250
Бейзлайн F1-Score: 0.7909, Улучшенный F1-Score: 0.8283, Custom GB F1-Score: 0.7006, Custom GB улучшенный F1-Score: 0.7253


---------------------

## Задача регрессии

In [40]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [41]:
path = 'hour.csv'
data_reg = pd.read_csv(path)

print("\nПервые 5 строк датасета для регрессии:")
print(data_reg.head())

# убрал дату в строковом представлении (есть дата в числовых колонках), так же число, которое предсказываем (cnt)
useful_columns = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 
                  'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']

# Разделим данные на тестовую и обучающую выборку
X = data_reg[useful_columns]
y = data_reg['cnt']

print(f"\nИспользуемые признаки: {list(X.columns)}")
print(f"Размер X: {X.shape}")


Первые 5 строк датасета для регрессии:
   instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1  

Используемые признаки: ['season', 'yr', 'mnth', 'hr', 'hol

### Разделяем данны и масштабируем

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape}, Размер тестовой выборки: {X_test.shape}")

scaler_reg = StandardScaler()
X_train_scaled = scaler_reg.fit_transform(X_train)
X_test_scaled = scaler_reg.transform(X_test)

Размер обучающей выборки: (13903, 12), Размер тестовой выборки: (3476, 12)


### Обучение бейзлайна и оценка модели

In [43]:
gb_baseline = GradientBoostingRegressor(random_state=42)
gb_baseline.fit(X_train_scaled, y_train)
y_pred_reg = gb_baseline.predict(X_test_scaled)

def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, r2

print("\nБейзлайн Gradient Boosting Regressor:")
mae_baseline, mse_baseline, r2_baseline = evaluate_model(y_test, y_pred_reg)
print(f"MAE: {mae_baseline:.4f}, MSE: {mse_baseline:.4f}, R²: {r2_baseline:.4f}")



Бейзлайн Gradient Boosting Regressor:
MAE: 47.1515, MSE: 4766.6716, R²: 0.8495


### Подберем гиперпараметры для улучшения модели


In [44]:
param_grid = {
    "n_estimators": [50, 100, 150],
    "learning_rate": [0.01, 0.1, 0.2],
    "max_depth": [3, 5, 7],
}

grid_search = GridSearchCV(
    GradientBoostingRegressor(random_state=42), 
    param_grid, 
    cv=5, 
    scoring="r2", 
    verbose=1
)
grid_search.fit(X_train_scaled, y_train)
best_params_reg = grid_search.best_params_
print(f"Лучшие параметры: {best_params_reg}")

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 150}


### Обучим модели с подобранными гиперпараметрами

In [45]:
gb_optimized = grid_search.best_estimator_
gb_optimized.fit(X_train_scaled, y_train)
y_pred_optimized = gb_optimized.predict(X_test_scaled)

mae_optimized, mse_optimized, r2_optimized = evaluate_model(y_test, y_pred_optimized)

print("\nУлучшенный Gradient Boosting Regressor:")
print(f"MAE: {mae_optimized:.4f}, MSE: {mse_optimized:.4f}, R²: {r2_optimized:.4f}")


Улучшенный Gradient Boosting Regressor:
MAE: 23.4889, MSE: 1543.0930, R²: 0.9513


### Реализуем свой алгоритм

In [46]:
class CustomGradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, min_samples_split=2):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.models = []
        self.base_prediction = None

    def fit(self, X, y):
        self.base_prediction = np.mean(y)
        residuals = y - self.base_prediction
        self.models = []
        
        for i in range(self.n_estimators):
            tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                random_state=42 + i
            )
            tree.fit(X, residuals)
            predictions = tree.predict(X)

            residuals -= self.learning_rate * predictions
            self.models.append(tree)

    def predict(self, X):
        predictions = np.full(X.shape[0], self.base_prediction)

        for tree in self.models:
            predictions += self.learning_rate * tree.predict(X)

        return predictions

### Посмотрим на результаты кастомной реализации с параметрами

In [47]:
custom_gb_reg = CustomGradientBoostingRegressor(
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=3
)
custom_gb_reg.fit(X_train_scaled, y_train)
y_pred_custom_reg = custom_gb_reg.predict(X_test_scaled)

mae_custom, mse_custom, r2_custom = evaluate_model(y_test, y_pred_custom_reg)

print("\nCustom Gradient Boosting Regressor (базовый):")
print(f"MAE: {mae_custom:.4f}, MSE: {mse_custom:.4f}, R²: {r2_custom:.4f}")

custom_gb_reg_improved = CustomGradientBoostingRegressor(
    n_estimators=best_params_reg['n_estimators'],
    learning_rate=best_params_reg['learning_rate'],
    max_depth=best_params_reg['max_depth'],
    min_samples_split=2
)
custom_gb_reg_improved.fit(X_train_scaled, y_train)
y_pred_custom_reg_improved = custom_gb_reg_improved.predict(X_test_scaled)

mae_custom_improved, mse_custom_improved, r2_custom_improved = evaluate_model(y_test, y_pred_custom_reg_improved)

print("\nCustom Gradient Boosting Regressor (улучшенный):")
print(f"MAE: {mae_custom_improved:.4f}, MSE: {mse_custom_improved:.4f}, R²: {r2_custom_improved:.4f}")


Custom Gradient Boosting Regressor (базовый):
MAE: 47.1493, MSE: 4766.5562, R²: 0.8495

Custom Gradient Boosting Regressor (улучшенный):
MAE: 23.4550, MSE: 1543.3993, R²: 0.9513


### Посмотрим на все результаты и сравним

In [48]:
print("\nСравнение результатов для задачи регрессии:")
print(f"Бейзлайн MAE: {mae_baseline:.4f}, Улучшенный MAE: {mae_optimized:.4f}, Custom GB MAE: {mae_custom:.4f}, Custom GB улучшенный MAE: {mae_custom_improved:.4f}")
print(f"Бейзлайн MSE: {mse_baseline:.4f}, Улучшенный MSE: {mse_optimized:.4f}, Custom GB MSE: {mse_custom:.4f}, Custom GB улучшенный MSE: {mse_custom_improved:.4f}")
print(f"Бейзлайн R²: {r2_baseline:.4f}, Улучшенный R²: {r2_optimized:.4f}, Custom GB R²: {r2_custom:.4f}, Custom GB улучшенный R²: {r2_custom_improved:.4f}")


Сравнение результатов для задачи регрессии:
Бейзлайн MAE: 47.1515, Улучшенный MAE: 23.4889, Custom GB MAE: 47.1493, Custom GB улучшенный MAE: 23.4550
Бейзлайн MSE: 4766.6716, Улучшенный MSE: 1543.0930, Custom GB MSE: 4766.5562, Custom GB улучшенный MSE: 1543.3993
Бейзлайн R²: 0.8495, Улучшенный R²: 0.9513, Custom GB R²: 0.8495, Custom GB улучшенный R²: 0.9513


----
## Подведем итог

### Задача классификации (Wine Quality Dataset)

| Алгоритм | Бейзлайн Accuracy | Улучшенный Accuracy | Custom Accuracy | Улучшенный Custom Accuracy |
|----------|-------------------|---------------------|-----------------|----------------------------|
| **KNN** | 0.7312 | 0.7500 | 0.7312 | 0.7500 |
| **Логистическая регрессия** | 0.7406 | 0.7375 | 0.7375 | 0.7406 |
| **Решающее дерево** | 0.7562 | 0.7594 | 0.7188 | 0.7531 |
| **Случайный лес** | 0.8063 | 0.8125 | 0.8000 | 0.7969 |
| **Градиентный бустинг** | 0.7906 | **0.8281** | 0.7063 | 0.7250 


### Задача регрессии (Bike Sharing Dataset)

| Алгоритм | Бейзлайн R² | Улучшенный R² | Custom R² | Улучшенный Custom R² |
|----------|-------------|---------------|-----------|----------------------|
| **KNN** | 0.9527 | 0.9514 | 0.9494 | 0.9514 |
| **Линейная регрессия** | **0.9666** | **0.9666** | **0.9666** | **0.9666** |
| **Случайный лес** | 0.9441 | 0.9449 | 0.9439 | 0.9447 |
| **Градиентный бустинг** | 0.8495 | 0.9513 | 0.8495 | 0.9513 |

### Эффективность алгоритмов
- **Градиентный бустинг** показал наилучшие результаты после настройки гиперпараметров
- **Ансамблевые методы** (Random Forest, Gradient Boosting) в целом превосходят одиночные алгоритмы
- **KNN** демонстрирует стабильную производительность на обоих типах задач

### Влияние настройки гиперпараметров
- Наибольший прирост от настройки: **Градиентный бустинг** (+0.0375 в классификации, +0.1018 в регрессии)
- Наименьший прирост: **Линейная регрессия** (минимальные изменения)
- **Custom реализации** в большинстве случаев уступают sklearn-версиям

### Выводы

1. **Для классификации** рекомендуется использовать **Gradient Boosting** с тщательной настройкой гиперпараметров
2. **Для регрессии** лучшие результаты показали **Linear Regression** и **Gradient Boosting**
3. **Custom реализации** в среднем на 2-10% уступают оптимизированным sklearn-версиям
4. **Ансамблевые методы** требуют значительной настройки, но дают лучшие результаты
5. **KNN** остается хорошим baseline-алгоритмом благодаря простоте и стабильности